In [1]:
import pandas as pd
import numpy as np
from IPython.display import HTML, display

In [2]:
# Upload de Datasets
# https://drive.google.com/drive/folders/1oncC_yOvZeNGmC_hLJeavdM4MPq3BkNd
# deben estar en la carpeta original_datasets

users_ff = pd.read_csv('original_datasets/users_ff.csv')
users_ss1 = pd.read_csv('original_datasets/users_ss1.csv')
users_ss2 = pd.read_csv('original_datasets/users_ss2.csv')
users_ss3 = pd.read_csv('original_datasets/users_ss3.csv')
users_ts = pd.read_csv('original_datasets/users_ts.csv')
users_humans = pd.read_csv('original_datasets/users_gen.csv')

# tweets_ff = pd.read_csv('original_datasets/tweets_ff.csv')
# tweets_ss1 = pd.read_csv('original_datasets/tweets_ss1.csv')
# tweets_ss2 = pd.read_csv('original_datasets/tweets_ss2.csv')
# tweets_ss3 = pd.read_csv('original_datasets/tweets_ss3.csv')
# tweets_ts = pd.read_csv('original_datasets/tweets_ts.csv')
tweets_humans = pd.read_csv('original_datasets/tweets_gen.csv')

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
users_humans.columns

Index(['id', 'name', 'screen_name', 'statuses_count', 'followers_count',
       'friends_count', 'favourites_count', 'listed_count', 'url', 'lang',
       'time_zone', 'location', 'default_profile', 'default_profile_image',
       'geo_enabled', 'profile_image_url', 'profile_banner_url',
       'profile_use_background_image', 'profile_background_image_url_https',
       'profile_text_color', 'profile_image_url_https',
       'profile_sidebar_border_color', 'profile_background_tile',
       'profile_sidebar_fill_color', 'profile_background_image_url',
       'profile_background_color', 'profile_link_color', 'utc_offset',
       'is_translator', 'follow_request_sent', 'protected', 'verified',
       'notifications', 'description', 'contributors_enabled', 'following',
       'created_at', 'timestamp', 'crawled_at', 'updated', 'Cat', 'bot'],
      dtype='object')

### Dataset Usuarios

In [4]:
# Agregar columna para categorizar
users_ss1.loc[:,'Cat'] = 'SocialSpambot'
users_ss2.loc[:,'Cat'] = 'SocialSpambot'
users_ss3.loc[:,'Cat'] = 'SocialSpambot'
users_ts.loc[:,'Cat'] = 'TraditionalSpambot'
users_ff.loc[:,'Cat'] = 'FollowersBot'
users_humans.loc[:,'Cat'] = 'Humans'

users_ss1.loc[:,'bot'] = True
users_ss2.loc[:,'bot'] = True
users_ss3.loc[:,'bot'] = True
users_ts.loc[:,'bot'] = True
users_ff.loc[:,'bot'] = True
users_humans.loc[:,'bot'] = False

In [5]:
# Identificar columnas que no coinciden entre datasets
total_columns = []
for x in [users_ss1, users_ss2, users_ss3, users_ts, users_ff, users_humans]:
#     print(x.columns.values)
    total_columns = np.concatenate((total_columns, x.columns.values))
    
total_columns = np.unique(np.array(total_columns))
dfHasColumns = pd.DataFrame( columns = total_columns)
for dfName in ['users_ss1', 'users_ss2', 'users_ss3', 'users_ts', 'users_ff', 'users_humans']:
    dataFrame = globals()[dfName]
    cols = dataFrame.columns.values
    hasCol = [1 if x in(cols) else 0 for x in dfHasColumns.columns.values]
    dfHasColumns.loc[dfName] = hasCol
dfHasColumns  = dfHasColumns.T
dfHasColumns.loc[:,'total'] = dfHasColumns.sum(axis=1)

dfHasColumns.sort_values(by='total').head(7)

,users_ss1,users_ss2,users_ss3,users_ts,users_ff,users_humans,total
test_set_2,0,0,1,0,0,1,2.0
test_set_1,1,0,0,0,0,1,2.0
crawled_at,1,1,1,1,0,1,5.0
timestamp,1,1,1,1,0,1,5.0
Cat,1,1,1,1,1,1,6.0
profile_background_tile,1,1,1,1,1,1,6.0
profile_banner_url,1,1,1,1,1,1,6.0


In [6]:
# for x in users_humans.columns.values:
#     display(
#         users_humans.loc[:,[x]][users_humans.loc[:,[x]].notnull().values].head(2)
#     )

#### Conclusiones de esta tabla:
La columna 'crawled_at' no se encuentra en users_ff (Folowers Bot).<br>
Esta columna es importante debido a que nos da una idea de la cantidad de meses que ese bot estuvo activo (y nos permite calcular una vida promedio del bot)
Hay tres opciones:<br>
1 No calcular la cantidad de meses antes que un bot se crea.<br>
2 No utilizar ese dataset.<br>
3 Asumir la fecha updated que esta presente en todos los casos

In [7]:
# Limpiar columnas innecesarias
users_ss1.drop(columns='test_set_1',inplace=True)
users_ss3.drop(columns='test_set_2',inplace=True)
users_humans.drop(columns=['test_set_1','test_set_2'],inplace=True)

In [8]:
# Concatenar datasets de Usuarios (SocialSpambots & Traditional Spambots)
users = pd.concat([users_ss1,users_ss2,users_ss3,users_ts,users_ff,users_humans],ignore_index=True)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [9]:
# Drop Cero Columns
users.drop(columns=['contributors_enabled','follow_request_sent','following','is_translator','notifications',],inplace=True)

In [10]:
tweets_humans.head()

,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,...,favorited,retweeted,possibly_sensitive,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated
0,593932392663912449,RT @morningJewshow: Speaking about Jews and co...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",678033.0,NaN,0.0,0.0,NaN,5.939322e+17,NaN,...,NaN,NaN,NaN,0.0,0.0,1.0,Fri May 01 00:18:11 +0000 2015,2015-05-01 02:18:11,2015-05-01 12:57:19,2015-05-01 12:57:19
1,593895316719423488,This age/face recognition thing..no reason pla...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",678033.0,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,Thu Apr 30 21:50:52 +0000 2015,2015-04-30 23:50:52,2015-05-01 12:57:19,2015-05-01 12:57:19
2,593880638069018624,Only upside of the moment I can think of is th...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",678033.0,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,2.0,0.0,0.0,Thu Apr 30 20:52:32 +0000 2015,2015-04-30 22:52:32,2015-05-01 12:57:19,2015-05-01 12:57:19
3,593847955536252928,If you're going to think about+create experien...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",678033.0,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,2.0,0.0,0.0,Thu Apr 30 18:42:40 +0000 2015,2015-04-30 20:42:40,2015-05-01 12:57:19,2015-05-01 12:57:19
4,593847687847350272,Watching a thread on FB about possible future ...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",678033.0,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,NaN,NaN,NaN,0.0,0.0,0.0,Thu Apr 30 18:41:36 +0000 2015,2015-04-30 20:41:36,2015-05-01 12:57:19,2015-05-01 12:57:19


In [44]:
display(tweets_humans.columns)

first = True
def getDataTweets(twU):
    global first
    if(first):
        dates = pd.to_datetime(twU.timestamp) #aca lo que hago es guardar la columnoa como datetime
        twU = twU.sort_values(by='timestamp', ascending=0)
        twU.loc[:,'dayOfWeek'] = dates.dt.dayofweek # dia de la semana
        twU.loc[:,'hour'] = dates.dt.hour #hora del tweet
        twU.loc[:,'weekNum'] = dates.apply(lambda x: str(x.week) + '-' + str(x.year))
        display(twU.head())
        
        times = pd.to_datetime(twU.timestamp.shift(1)) - pd.to_datetime(twU.timestamp) # calculo distancia entre tweets
        display(times.median())
    first = False

tweets_humans.groupby('user_id').apply(getDataTweets)

Index(['id', 'text', 'source', 'user_id', 'truncated', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'in_reply_to_screen_name', 'retweeted_status_id',
       'geo', 'place', 'contributors', 'retweet_count', 'reply_count',
       'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive',
       'num_hashtags', 'num_urls', 'num_mentions', 'created_at', 'timestamp',
       'crawled_at', 'updated'],
      dtype='object')

,id,text,source,user_id,truncated,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,retweeted_status_id,geo,...,num_hashtags,num_urls,num_mentions,created_at,timestamp,crawled_at,updated,dayOfWeek,hour,weekNum
0,593932392663912449,RT @morningJewshow: Speaking about Jews and co...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",678033.0,NaN,0.0,0.0,NaN,5.939322e+17,NaN,...,0.0,0.0,1.0,Fri May 01 00:18:11 +0000 2015,2015-05-01 02:18:11,2015-05-01 12:57:19,2015-05-01 12:57:19,4,2,18-2015
1,593895316719423488,This age/face recognition thing..no reason pla...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",678033.0,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,0.0,0.0,0.0,Thu Apr 30 21:50:52 +0000 2015,2015-04-30 23:50:52,2015-05-01 12:57:19,2015-05-01 12:57:19,3,23,18-2015
2,593880638069018624,Only upside of the moment I can think of is th...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",678033.0,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,2.0,0.0,0.0,Thu Apr 30 20:52:32 +0000 2015,2015-04-30 22:52:32,2015-05-01 12:57:19,2015-05-01 12:57:19,3,22,18-2015
3,593847955536252928,If you're going to think about+create experien...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",678033.0,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,2.0,0.0,0.0,Thu Apr 30 18:42:40 +0000 2015,2015-04-30 20:42:40,2015-05-01 12:57:19,2015-05-01 12:57:19,3,20,18-2015
4,593847687847350272,Watching a thread on FB about possible future ...,"<a href=""http://tapbots.com/tweetbot"" rel=""nof...",678033.0,NaN,0.0,0.0,NaN,0.000000e+00,NaN,...,0.0,0.0,0.0,Thu Apr 30 18:41:36 +0000 2015,2015-04-30 20:41:36,2015-05-01 12:57:19,2015-05-01 12:57:19,3,20,18-2015


Timedelta('0 days 00:05:11.500000')

""


### Dataset Tweets

In [31]:
# Agregar columna para categorizar
tweets_ss1['Cat'] = 'SocialSpambot'
tweets_ss2['Cat'] = 'SocialSpambot'
tweets_ss3['Cat'] = 'SocialSpambot'
tweets_ts['Cat'] = 'TraditionalSpambot'
tweets_ff['Cat'] = 'FollowersBot'
tweets_humans['Cat'] = 'Humans'
# Concatenar datasets de Usuarios (SocialSpambots & Traditional Spambots)
tweets = pd.concat([tweets_ss1,tweets_ss2,tweets_ss3,tweets_ts,tweets_ff,tweets_humans],ignore_index=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


### Analizar correspondencia entre datasets

In [40]:
print('Usuarios unicos en dataset de Users: ',users.id.nunique())
print('Usuarios unicos en dataset de Tweets: ',tweets.user_id.nunique())

Usuarios unicos en dataset de Users:  12737
Usuarios unicos en dataset de Tweets:  10197


In [35]:
# Crear tabla de cuenta de tweets desde dataset de tweets
id_count_tweets = tweets[['user_id','id']].groupby('user_id',as_index=False).count()
id_count_tweets.rename(columns={'user_id':'id','id':'tweet_count'}, inplace=True)
# Joinear datasets para agregar a Users la cuenta tweets
users = users.merge(id_count_tweets,how='left',on='id')

In [44]:
print('Cantidad de usuarios sin tweets: ',users.tweet_count.isna().sum())

Cantidad de usuarios sin tweets:  2540


In [51]:
# Explorar usuarios sin tweets por categoria
users[['Cat','id']][users.tweet_count.isna()].groupby('Cat').count()

,id
Cat,
FollowersBot,149
Humans,2391


In [59]:
print('Cantidad de usuarios Humanos únicos en el dataset original: ',users[users.Cat=="Humans"].id.nunique())
print('Proporcion de usuarios humanos que tienen tweets: ',tweets[tweets.Cat=="Humans"].user_id.nunique()/users[users.Cat=="Humans"].id.nunique())

Cantidad de usuarios Humanos únicos en el dataset original:  3474
Proporcion de usuarios humanos que tienen tweets:  0.31174438687392053
